In [ ]:
%%capture
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [ ]:
!pip install -q -U bitsandbytes

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
read_key = user_secrets.get_secret("HF_READ")

In [ ]:
from huggingface_hub import login
login(token= read_key)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

if torch.cuda.is_bf16_supported():
    compute_dtype = torch.bfloat16
else:
    compute_dtype = torch.float16

model_name = "JefiRyan/Gemma-2B-Unsloth-mental-health-merged"
quant_path = 'gemma-2b-4bit-quantized'

tokenizer = AutoTokenizer.from_pretrained(model_name)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
    
model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config
)

In [ ]:
model.save_pretrained("./BnB"+quant_path, safetensors = True)
tokenizer.save_pretrained("./BnB"+quant_path)